In [98]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [103]:
CWD = os.getcwd()
NOW = datetime.now().strftime('%m/%d/%Y, %H:%M:%S')

In [14]:
def get_current_currency_rate(currency_from: str, currency_to: str) -> float:
    """
    This method is used to scrap real time currency from x-rates.com. This method return us rate of given comparison
    @param currency_from: <str> currency from
    @param currency_to: <str> currency to
    return rate: <float> rate of target currency
    """
    currency_from = currency_from.upper()
    currency_to = currency_to.upper()
    request_url: str = f"https://www.x-rates.com/calculator/?from={currency_from}&to={currency_to}&amount=1"
    with requests.Session() as session:
        response = session.get(url=request_url)
        soup_obj = BeautifulSoup(response.text, "html.parser")

    part1 = soup_obj.find(class_="ccOutputTrail").previous_sibling
    part2 = soup_obj.find(class_="ccOutputTrail").get_text(strip=True)
    rate = "{}{}".format(part1, part2)
    return float(rate)

In [15]:
get_current_currency_rate('USD', 'TRY')

23.654252

In [109]:
def get_currency_by_day(currency_from: str, date: str):
    df = []
    c_from = currency_from.upper()
    date = date.upper()
    request_url: str = f"https://www.x-rates.com/historical/?from={c_from}&amount=1&date={date}"
    with requests.Session() as session:
        response = session.get(url=request_url,
                               headers={'Content-Type': 'text/html'})
        soup_obj = BeautifulSoup(response.text, "html.parser")
    # parse html
    table = soup_obj.find('table', {
        'class': 'ratesTable'
    }).find('tbody').find_all('tr')
    for item in table:
        data = item.find_all('td')
        currency_name = data[0].text
        currency_amount = data[1].find('a').text
        inverse_currency_amount = data[2].find('a').text
        df.append({
            'from_currency': c_from,
            'to_currency': currency_name,
            'currency_amount': currency_amount,
            'inverse_currency_amount': inverse_currency_amount,
            'date': date,
            'created_at': NOW
        })
    return pd.DataFrame(df)


get_currency_by_day(currency_from='TRY', date='2020-06-14')

,from_currency,to_currency,currency_amount,inverse_currency_amount,date,created_at
0,TRY,US Dollar,0.146771,6.813354,2020-06-14,"06/16/2023, 00:36:16"
1,TRY,Euro,0.130416,7.667746,2020-06-14,"06/16/2023, 00:36:16"
2,TRY,British Pound,0.117011,8.546208,2020-06-14,"06/16/2023, 00:36:16"
3,TRY,Indian Rupee,11.146868,0.089711,2020-06-14,"06/16/2023, 00:36:16"
4,TRY,Australian Dollar,0.213979,4.673353,2020-06-14,"06/16/2023, 00:36:16"
5,TRY,Canadian Dollar,0.199347,5.016386,2020-06-14,"06/16/2023, 00:36:16"
6,TRY,Singapore Dollar,0.204374,4.892991,2020-06-14,"06/16/2023, 00:36:16"
7,TRY,Swiss Franc,0.139750,7.155657,2020-06-14,"06/16/2023, 00:36:16"
8,TRY,Malaysian Ringgit,0.626803,1.595397,2020-06-14,"06/16/2023, 00:36:16"
9,TRY,Japanese Yen,15.760935,0.063448,2020-06-14,"06/16/2023, 00:36:16"


In [114]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


def get_currency_by_day(currency_from: str, date: str):
    c_from = currency_from.upper()
    request_url = f"https://www.x-rates.com/historical/?from={c_from}&amount=1&date={date}"

    with requests.Session() as session:
        response = session.get(url=request_url,
                               headers={'Content-Type': 'text/html'})

    soup_obj = BeautifulSoup(response.text, "html.parser")

    # parse html
    rows = soup_obj.select('table.ratesTable tbody tr')
    df = [{
        'from_currency': c_from,
        'to_currency': row.select_one('td:nth-of-type(1)').text,
        'currency_amount': row.select_one('td:nth-of-type(2) a').text,
        'inverse_currency_amount': row.select_one('td:nth-of-type(3) a').text,
        'date': date,
        'created_at': NOW
    } for row in rows]

    return pd.DataFrame(df).drop_duplicates()


get_currency_by_day(currency_from='USD', date='2020-06-14')

,from_currency,to_currency,currency_amount,inverse_currency_amount,date,created_at
0,USD,Euro,0.888573,1.125400,2020-06-14,"06/16/2023, 00:36:16"
1,USD,British Pound,0.797237,1.254332,2020-06-14,"06/16/2023, 00:36:16"
2,USD,Indian Rupee,75.947557,0.013167,2020-06-14,"06/16/2023, 00:36:16"
3,USD,Australian Dollar,1.457916,0.685911,2020-06-14,"06/16/2023, 00:36:16"
4,USD,Canadian Dollar,1.358219,0.736258,2020-06-14,"06/16/2023, 00:36:16"
5,USD,Singapore Dollar,1.392472,0.718147,2020-06-14,"06/16/2023, 00:36:16"
6,USD,Swiss Franc,0.952163,1.050240,2020-06-14,"06/16/2023, 00:36:16"
7,USD,Malaysian Ringgit,4.270633,0.234157,2020-06-14,"06/16/2023, 00:36:16"
8,USD,Japanese Yen,107.384826,0.009312,2020-06-14,"06/16/2023, 00:36:16"
9,USD,Chinese Yuan Renminbi,7.081075,0.141221,2020-06-14,"06/16/2023, 00:36:16"
